# Use GPT-4 API and Zero Shot Learning to produce the topic at paragraph level.

In [18]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
import pandas as pd
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.chat_models import ChatOpenAI
import re
import time
from config import *
from dotenv import load_dotenv
load_dotenv()

True

### Create zero shot prompt using Langchain

In [8]:
# Create instruction for the task
pre_fix_template="""你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：{topic_list}。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。""".format(topic_list = ','.join(topics_pdf))

In [9]:
print(pre_fix_template)

你需要给一段文字打上话题标签。我会给你话题列表和文字内容。话题必须从给定的话题列表里选出，而且只能给一个话题。
话题列表是：基建,房地产,制造业,科技,零售消费,绿色发展,医药健康,教育,交通运输,出口贸易,农业农村,文化旅游,就业,民生扶贫,能源,物资储备,宏观,区域发展,企业发展,新冠疫情,地方财政,税务,国际关系,金融。
你的回答的格式必须是：回答-话题
请给出这段文字的话题。你的回答仅包括话题。你给出的话题必须是话题列表里面的一个，而且只能给一个话题。


In [36]:
# Create zero shot prompt template
zero_prompt = pre_fix_template + "\n文字内容：{content}"

### Run few shot learning on whole dataset

In [38]:
# Initate Langchain with GPT-4 API
llm = ChatOpenAI(temperature = 0, openai_api_key = OPENAI_API_KEY, model_name = 'gpt-4')

llm_chain = LLMChain(
    llm = llm,
    prompt = PromptTemplate.from_template(zero_prompt)
)

In [39]:
# Load whole dataset
df = pd.read_csv(data_path)

In [40]:
df.head()

,ID,RAW_CONTENT,MEETING_NAME,TITLE,PUBLISH_DATE
0,m-1608357555-1682084460,中共中央总书记、国家主席、中央军委主席、中央全面深化改革委员会主任习近平4月21日下午主...,中央全面深化改革会议,会议审议通过了关于强化企业科技创新主体地位的意见关于加强和改进国有经济管理有力支持中国式现代...,2023-04-21T21:41:00Z
1,m-158629831-1682664300,中共中央政治局4月28日召开会议，分析研究当前经济形势和经济工作。中共中央总书记习近平主...,政治局会议,分析研究当前经济形势和经济工作,2023-04-28T14:45:00Z
2,m-704972677-1680162720,中共中央政治局3月30日召开会议，决定从今年4月开始，在全党自上而下分两批开展学习贯彻习...,政治局会议,决定从2023年4月开始在全党自上而下分两批开展学习贯彻习近平新时代中国特色社会主义思想主题...,2023-03-30T15:52:00Z
3,m-248496144-1366366560,中共中央政治局4月19日召开会议，决定从今年下半年开始，用一年左右时间，在全党自上而下分...,政治局会议,研究部署在全党深入开展党的群众路线教育实践活动,2013-04-19T18:16:00Z
4,m-1020277137-1469618520,李克强主持召开国务院常务会议\n听取关于地方和部门推进重大项目落地审计情况汇报 完善奖惩机制...,国务院常委会,李克强主持召开国务院常务会议通过十三五国家科技创新专项规划 以创新型国家建设引领和支撑升级发...,2016-07-27T19:22:00Z


In [16]:
df.shape

(531, 5)

In [43]:
id_list = []
title_list = []
content_list = []
topic_list = []
ln_min = 50  # Paragraphs shorter than 50 characters will be ignored
cnt = 0

In [47]:
for index, row in df.iterrows():
    txt = re.split('\u3000\u3000|\r\n', row['RAW_CONTENT']) # Split the content into paragraphs
    txt = [x.strip() for x in txt]
    for x in txt:
        if len(x) >= ln_min:
            id_list.append(row['ID'])
            title_list.append(row['TITLE'])
            content_list.append(x)

In [48]:
df_para = pd.DataFrame({'ID': id_list, 
                        'TITLE': title_list,
                        'CONTENT': content_list
                       })

In [49]:
df_para.shape

(3540, 3)

In [50]:
df_para.to_csv('data/all_para.csv', index = False, encoding='utf_8_sig')

In [44]:
for index, row in df.iterrows():
    txt = re.split('\u3000\u3000|\r\n', row['RAW_CONTENT']) # Split the content into paragraphs
    txt = [x.strip() for x in txt]
    for x in txt:
        if len(x) >= ln_min:
            id_list.append(row['ID'])
            title_list.append(row['TITLE'])
            content_list.append(x)
            topic_list.append(llm_chain.predict(content=x).split('话题：')[-1])
            cnt += 1
            if cnt % 100 == 0:
                print(str(cnt) + ' paragraphs have been processed...')

100 paragraphs have been processed...
200 paragraphs have been processed...
300 paragraphs have been processed...
400 paragraphs have been processed...


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 2.0 seconds as it raised RateLimitError: You exceeded your current quota, please check your plan and billing details..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_ret

RateLimitError: You exceeded your current quota, please check your plan and billing details.

In [ ]:
df_para_few = pd.DataFrame({'ID': id_list, 
                               'TITLE': title_list,
                               'CONTENT': content_list,
                              'TOPIC': topic_list})

In [ ]:
df_para_few.to_csv('data/all_para_few.csv', index = False, encoding='utf_8_sig')

In [45]:
index

43

In [46]:
topic_list

['回答-宏观',
 '回答-企业发展',
 '回答-科技',
 '回答-企业发展',
 '回答-企业发展',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-科技',
 '回答-宏观',
 '回答-企业发展',
 '回答-出口贸易',
 '回答-房地产',
 '回答-民生扶贫',
 '回答-教育',
 '回答-教育',
 '回答-教育',
 '回答-教育',
 '回答-教育',
 '回答-企业发展',
 '回答-企业发展',
 '回答-宏观',
 '回答-教育',
 '回答-教育',
 '回答-教育',
 '回答-民生扶贫',
 '回答-教育',
 '回答-教育',
 '回答-民生扶贫',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-科技',
 '回答-科技',
 '回答-科技',
 '回答-绿色发展',
 '回答-农业农村',
 '回答-医药健康',
 '回答-科技',
 '回答-科技',
 '回答-科技',
 '回答-区域发展',
 '回答-科技',
 '回答-金融',
 '回答-宏观',
 '回答-科技',
 '回答-科技',
 '回答-地方财政',
 '回答-绿色发展',
 '回答-金融',
 '回答-科技',
 '回答-企业发展',
 '回答-宏观',
 '回答-企业发展',
 '回答-企业发展',
 '回答-金融',
 '回答-教育',
 '回答-企业发展',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-区域发展',
 '回答-政府监管',
 '回答-教育',
 '回答-科技',
 '回答-民生扶贫',
 '回答-科技',
 '回答-科技',
 '回答-科技',
 '回答-科技',
 '回答-能源',
 '回答-教育',
 '回答-文化旅游',
 '回答-科技',
 '回答-宏观',
 '回答-宏观',
 '回答-企业发展',
 '回答-企业发展',
 '回答-物资储备',
 '回答-物资储备',
 '回答-绿色发展',
 '回答-绿色发展',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-宏观',
 '回答-农业农村